In [1]:
from sqlalchemy import create_engine
import db_params

In [2]:
import os, pickle

In [3]:
import pandas as pd

db_kwargs = db_params.get_db_params(
        db_name="gwgdb", 
        db_type="postgresql"
    )

In [5]:
file_pkl = os.path.join("data", f"yfinance-fa-data.pkl")
with open(file_pkl, "rb") as f:
    fa_data = pickle.load(f)

In [6]:
len(fa_data)

5142

In [7]:
symbols = [r[0] for r in fa_data]

In [8]:
len(symbols)

5142

In [9]:
len(set(symbols))

5142

In [10]:
key_attribs = [
    "symbol", "sector", "industry", "shortName",
    "website", "phone", "address1", "city", "state", "zip", "country", 
    "fullTimeEmployees", 
    "logo_url", 
    "longBusinessSummary",
    "dividendYield",
    "forwardPE", "forwardEps", "trailingEps", 
    "bookValue", 
    "sharesOutstanding", "sharesShort"
]

In [11]:
key2idx_map = {}
for i,k in enumerate(key_attribs):
    key2idx_map[k] = i

In [12]:
column_types = {
    "symbol": "character varying ", 
    "sector": "character varying ",
    "industry": "character varying ", 
    "name": "character varying ",
    # "comments": "character varying ",
    "website": "character varying ", 
    "num_emp": "integer", 
    "div_yield": "numeric",
    "fpe": "numeric", 
    "feps": "numeric", 
    "teps": "numeric", 
    "book_val": "numeric", 
    "shares": "integer", 
    "shares_short": "integer",
    "profile": "text",
    "phone": "character varying ", 
    "address1": "character varying ", 
    "city": "character varying ", 
    "state": "character varying ", 
    "zip": "character varying ", 
    "country": "character varying ", 
    "logo_url": "character varying "
}

In [13]:
key2col_map = {'symbol': 'symbol',
 'sector': 'sector',
 'industry': 'industry',
 'shortName': 'name',
 'website': 'website',
 'phone': 'phone',
 'address1': 'address1',
 'city': 'city',
 'state': 'state',
 'zip': 'zip',
 'country': 'country',
 'fullTimeEmployees': 'num_emp',
 'logo_url': 'logo_url',
 'longBusinessSummary': 'profile',
 'dividendYield': 'div_yield',
 'forwardPE': 'fpe',
 'forwardEps': 'feps',
 'trailingEps': 'teps',
 'bookValue': 'book_val',
 'sharesOutstanding': 'shares',
 'sharesShort': 'shares_short'}

In [14]:
col2key_map = {}
for k,v in key2col_map.items():
    col2key_map[v] = k

In [15]:
columns = column_types.keys()

In [16]:
len(columns), columns

(21,
 dict_keys(['symbol', 'sector', 'industry', 'name', 'website', 'num_emp', 'div_yield', 'fpe', 'feps', 'teps', 'book_val', 'shares', 'shares_short', 'profile', 'phone', 'address1', 'city', 'state', 'zip', 'country', 'logo_url']))

In [17]:
engine = create_engine(f'postgresql://{db_kwargs["user"]}:{db_kwargs["password"]}@{db_kwargs["host"]}:{db_kwargs["port"]}/{db_kwargs["database"]}')

In [18]:
conn = engine.connect()
df_fa = pd.DataFrame(columns = columns)
for row in fa_data:
    record = [row[key2idx_map[col2key_map[c]]] for c in columns]
    df_fa = df_fa.append(
        pd.Series(record, index=columns),
        ignore_index = True
    )
    
# https://datatofish.com/pandas-dataframe-to-sql/
df_fa.to_sql('ticker', conn, if_exists='replace', index=False)
    
conn.close()